Before proceeding with batch processing, the cost should be estimated. To do so, the batch processing file will be used together with `hugging-face`. Tokenizer for for `llama-3.1-8b` will be used.

In [1]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.1-8B")

/home/oleksii/python/embed_steer/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import json 

with open('../test-batch-syn-queries.jsonl', 'r') as file: 
    content = file.read() 
    calls = content.split('\n') 
    calls = list(map(lambda x: json.loads(x), calls)) 

calls[0]

{'custom_id': 'doc0',
 'method': 'POST',
 'url': '/v1/chat/completions',
 'body': {'model': 'llama-3.1-8b-instant',
  'messages': [{'role': 'system',
    'content': 'You are a helpful AI assistant.\nGenerate most relevant and short search question for the document excerpt provided by the user.\nReturn only the generated query without quotes or trailing punctuation marks.'},
   {'role': 'user',
    'content': "Minority interest In accounting, minority interest (or non-controlling interest) is the portion of a subsidiary corporation's stock that is not owned by the parent corporation. The magnitude of the minority interest in the subsidiary company is generally less than 50% of outstanding shares, or the corporation would generally cease to be a subsidiary of the parent.[1]"}]}}